# Prepare TESS data for Cotrendy

Take in a list of TESS data from Sam's FFI analysis and make the pickled inputs for Cotrendy

In [3]:
%matplotlib inline
import pickle
import numpy as np
import glob as g
import os
from astropy.io import fits
import matplotlib.pyplot as plt

In [4]:
def picklify(pickle_file, pickle_object):     
    """                                       
    Take an object and pickle it for later use
    """                                       
    of = open(pickle_file, 'wb')              
    pickle.dump(pickle_object, of)            
    of.close()  

In [5]:
data_dir = "/Users/jmcc/Dropbox/PythonScripts/TESS/mono_example_data"

mask_file = f"{data_dir}/S05_mask.fits"
m = fits.open(mask_file)
mask = m[1].data['MASK']

cat_file = f"{data_dir}/S05_1-1.fits"
cat = fits.open(cat_file)[1].data

os.chdir(data_dir)
tic_files = sorted(g.glob('TIC-*.fits'))

fluxes_to_trendy, ras, decs, mags = [], [], [], [] 
neg_fluxes = []
dilutions = []
tic_files_kept = []

for tic_file in tic_files:
    
    tic_id = int(tic_file.split('.fits')[0].split('-')[1])
    
    h = fits.open(tic_file)[1].data
    flux = h['AP2.5'][mask]
    sky = h['SKY_MEDIAN'][mask] * np.pi * (2.5**2)
    flux_corr = flux - sky
    neg = np.sum([flux_corr<0])
    if neg == 0:
        # make a catalog cut first
        # get the catalog info for pickle file
        row_in_cat = np.where(cat['TIC_ID'] == tic_id)[0]
        
        ra = cat['RA'][row_in_cat][0]
        dec = cat['DEC'][row_in_cat][0]
        mag = cat['Tmag'][row_in_cat][0]
        
        # the finat ones slow everything down and likely don't correlate well anyway
        # try removing them and see what the CBVs look like. 
        if mag > 8 and mag < 12.0:
            ras.append(ra)
            decs.append(dec)
            mags.append(mag)
            tic_files_kept.append(tic_file)
            fluxes_to_trendy.append(flux_corr)
            times0 = int(h['BJD'][mask][0])
            times = h['BJD'][mask] - times0
    else:
        neg_fluxes.append(tic_file)
            
fluxes_to_trendy = np.array(fluxes_to_trendy)

In [ ]:
fig, ax = plt.subplots(3, figsize=(10, 10))
ax[0].hist(ras, bins=100)
ax[1].hist(decs, bins=100)
ax[2].hist(mags, bins=100)
plt.show()

In [6]:
neg_fluxes

['TIC-10996672.fits',
 'TIC-248620783.fits',
 'TIC-248657949.fits',
 'TIC-250135660.fits',
 'TIC-34362037.fits',
 'TIC-454140671.fits',
 'TIC-454140677.fits',
 'TIC-454140679.fits',
 'TIC-454143519.fits',
 'TIC-454154422.fits',
 'TIC-9236522.fits']

In [ ]:
tic_files_kept.index('TIC-459932301.fits')

In [ ]:
tic_files_kept.index('TIC-9161351.fits')

In [ ]:
# pickle the outputs
picklify('tess_s05_times.pkl', times)
picklify('tess_s05_fluxes.pkl', fluxes_to_trendy)
picklify('tess_s05_errors.pkl', np.sqrt(fluxes_to_trendy))

# pickle the catalog into a 2D numpy array
picklify('tess_s05_cat.pkl', np.array([ras, decs, mags]))


In [ ]:
def depicklify(pickle_file):                   
    """                                        
    Take a pickled file and return the object  
    """                                        
    try:                                       
        of = open(pickle_file, 'rb')           
        res = pickle.load(of)                  
        of.close()                             
    except FileNotFoundError:                  
        res = None                             
    return res                                 

In [ ]:
time_dp = depicklify('tess_s05_times.pkl')
fluxes_dp = depicklify('tess_s05_fluxes.pkl')
errors_dp = depicklify('tess_s05_errors.pkl')

In [ ]:
time_dp

In [ ]:
fluxes_dp

In [ ]:
errors_dp

In [ ]:
picklify('tic_ids_kept.pkl', tic_files_kept)